<a href="https://colab.research.google.com/github/dhruvchopra2003/Paper2/blob/main/LLM_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-08 18:49:26--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-09-08 18:49:26 (46.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 16 # how many independent sequences will we process in
block_size = 32 # maximum context length for
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

#_________________

torch.manual_seed(1337)
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Here we are trying to create a character level language
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Create mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]

decode = lambda l: ''.join([itos[i] for i in l])

# train test split
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# Data loading
def get_batch(split):
  # Generate small batches of data of inputs x and y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  # This is being done to push the tensors onto the GPU to accelerate training
  x, y = x.to(device), y.to(device)

  return x, y


#This is basically to optimize pytorch. It tells that whatever intermediate
# variables are created, don't store them, coz we're never gonna call backwards
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

# The bigram language model
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads from the logits for the next token from the lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    # idx and targets are both (B, T) tensor of integers
    logits = self.token_embedding_table(idx) # (B, T, C)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current
    for _ in range(max_new_tokens):
      # get the prediction
      logits, loss = self(idx)
      # Focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # Apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # Applied sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# Create pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
  xb, yb = get_batch('train')
  # evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

# Generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))



step 0: train loss 4.7260, val loss 4.7259
step 100: train loss 4.5986, val loss 4.5953
step 200: train loss 4.4763, val loss 4.4755
step 300: train loss 4.3572, val loss 4.3574
step 400: train loss 4.2446, val loss 4.2459
step 500: train loss 4.1332, val loss 4.1303
step 600: train loss 4.0279, val loss 4.0317
step 700: train loss 3.9268, val loss 3.9321
step 800: train loss 3.8344, val loss 3.8387
step 900: train loss 3.7425, val loss 3.7476
step 1000: train loss 3.6578, val loss 3.6587
step 1100: train loss 3.5823, val loss 3.5821
step 1200: train loss 3.5049, val loss 3.5055
step 1300: train loss 3.4314, val loss 3.4358
step 1400: train loss 3.3648, val loss 3.3694
step 1500: train loss 3.2985, val loss 3.3034
step 1600: train loss 3.2409, val loss 3.2487
step 1700: train loss 3.1900, val loss 3.1855
step 1800: train loss 3.1357, val loss 3.1408
step 1900: train loss 3.0850, val loss 3.0887
step 2000: train loss 3.0422, val loss 3.0514
step 2100: train loss 3.0007, val loss 3.0041


## Mathematical Logic behind self attention

In [10]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

Now we need the tokens to interact with each other. We basically want each token to have some representation in terms of it's preceeding tokens. Simplest way is to take the average of the channels of each predecessor.

In [15]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) # bow aka: bag of words -> used for averaging a set of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [48]:
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [16]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [18]:
xbow[0] # each row is the average of all the elements above it (in the same column)
# This implies that the last row has the average of the entire set

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [41]:
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=False)
a

tensor([[1.0000, 0.0000, 0.0000],
        [1.0000, 0.5000, 0.0000],
        [1.0000, 0.5000, 0.3333]])

In [43]:
# Now we can be much more efficient doing this by using matrix multiplication
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [45]:
# In the above code, after using multiplying tensor b with the row averaged lower triangular matrix, we find that in tensor c, each row is the average of the rows above it.
# This means that c[n-1] row has the average of all the rows before it

In [57]:
# Now let's create use this averaging logic (using a matrix a [lower triangular matrix with row averaged weights]) and apply it to vectorize our bag of words tensor before
# This would help us to associate each word with all it's predecessors

# The lower triangular matrix actually ensures that each element only gets information from it's predecessors and not it's successors.

In [56]:
wei = torch.tril(torch.ones(T, T))
wei = wei/torch.sum(wei, 1, keepdim=True)
print(f"Weight matrix: \n{wei}") # each row of it sums to 1

# our b here is: x
print(f"  Input matrix: \n{x}")

# Notice that there is a slight difference in the dimensions. X has dimensions: (B, T, C) however wei has dimensions (T, T). Torch manages itself and creates batches (B) for
# the wei matrix to be able to multiply correctly

xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)
torch.allclose(xbow, xbow2, rtol=1e-04, atol=1e-06)

Weight matrix: 
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
  Input matrix: 
tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.56

True

In [58]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
# torch.allclose(xbow, xbow3)
torch.allclose(xbow, xbow3, rtol=1e-04, atol=1e-06)



True

## Making some small changes to the script

In [59]:
""" Changes:
  -> remove vocab size from the BigramLanguageModel constructor as it is already defined.
  -> Introducing a new variable n_embd: short for number of embedding dimensions
"""
import torch
import torch.nn as nn
from torch.nn import functional as F

# Hyperparameters
batch_size = 16 # how many independent sequences will we process in
block_size = 32 # maximum context length for
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32 # number of embedding dimensions

#__________________________________________________________________________

torch.manual_seed(1337)
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Here we are trying to create a character level language
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Create mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]

decode = lambda l: ''.join([itos[i] for i in l])

# train test split
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# Data loading
def get_batch(split):
  # Generate small batches of data of inputs x and y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  # This is being done to push the tensors onto the GPU to accelerate training
  x, y = x.to(device), y.to(device)

  return x, y


#This is basically to optimize pytorch. It tells that whatever intermediate
# variables are created, don't store them, coz we're never gonna call backwards
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


# _______________________________________________________________________________________

# The bigram language model
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    # each token directly reads from the logits for the next token from the lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size) # This layer is used to go from token embeddings to the logits
    # LM_head stands for language model head

  def forward(self, idx, targets=None):
    # idx and targets are both (B, T) tensor of integers

    # When we replace the vocab size with n_embd, it wouldn't give us logits directly, but rather token embeddings
    tok_emb = self.token_embedding_table(idx) # (B, T, C)
    logits = self.lm_head(tok_emb) # (B, T, vocab_size())

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current
    for _ in range(max_new_tokens):
      # get the prediction
      logits, loss = self(idx)
      # Focus only on the last time step
      logits = logits[:, -1, :] # becomes (B, C)
      # Apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # Applied sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

# _____________________________________________________________________________

# Create model
model = BigramLanguageModel()
m = model.to(device)

# Create pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  # every once in a while evaluate the loss on train and val sets
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
  xb, yb = get_batch('train')
  # evaluate the loss
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

# Generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))



step 0: train loss 4.3857, val loss 4.3773
step 100: train loss 3.5081, val loss 3.5110
step 200: train loss 3.0604, val loss 3.0634
step 300: train loss 2.8407, val loss 2.8571
step 400: train loss 2.7288, val loss 2.7438
step 500: train loss 2.6491, val loss 2.6719
step 600: train loss 2.6159, val loss 2.6350
step 700: train loss 2.5908, val loss 2.6085
step 800: train loss 2.5589, val loss 2.5747
step 900: train loss 2.5479, val loss 2.5706
step 1000: train loss 2.5414, val loss 2.5546
step 1100: train loss 2.5327, val loss 2.5480
step 1200: train loss 2.5229, val loss 2.5386
step 1300: train loss 2.5233, val loss 2.5298
step 1400: train loss 2.5080, val loss 2.5243
step 1500: train loss 2.5060, val loss 2.5102
step 1600: train loss 2.5112, val loss 2.5210
step 1700: train loss 2.4974, val loss 2.5086
step 1800: train loss 2.4995, val loss 2.5159
step 1900: train loss 2.4898, val loss 2.5111
step 2000: train loss 2.4965, val loss 2.5130
step 2100: train loss 2.4911, val loss 2.5068


In [8]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [9]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)